In [2]:
import numpy as np 
import pandas as pd 
#df = pd.read_csv("../Analyzed_Games/twic1556_15_analyzed.csv")
#df=pd.read_csv("../Analyzed_Games/test2_15_analyzed.csv")
#df= pd.read_csv("../Analyzed_Games/twic920_15_analyzed.csv")


# Cleaning data (use Dorian's cleaning function instead of this for better results)

In [4]:
initial_game_count = df['GameID'].nunique()

# Step 1: Identify GameIDs with valid 'Result'
valid_result_games = df[df['Result'].isin(['1-0', '0-1', '1/2-1/2'])]['GameID'].unique()

# Step 2: Identify GameIDs with no missing 'WhiteFideId' or 'BlackFideId'
fide_valid_games = df.dropna(subset=['WhiteFideId', 'BlackFideId'])['GameID'].unique()

# Step 3: Find the intersection of valid games
valid_games = np.intersect1d(valid_result_games, fide_valid_games)

# Step 4: Filter the DataFrame to include only valid games
df_cleaned = df[df['GameID'].isin(valid_games)].copy()

# Record the final number of unique games
final_game_count = df_cleaned['GameID'].nunique()

# Calculate the number of games removed
removed_games = initial_game_count - final_game_count

# Reset the index
df_cleaned = df_cleaned.reset_index(drop=True)

# Create a mapping from old GameID to new sequential GameID
unique_games = df_cleaned['GameID'].unique()
game_id_mapping = {old_id: new_id for new_id, old_id in enumerate(unique_games, start=1)}

# Apply the mapping to fix 'GameID'
df_cleaned['GameID'] = df_cleaned['GameID'].map(game_id_mapping)

# Save the cleaned DataFrame to a new CSV file (optional)
# df_cleaned.to_csv("../huge_analyzed_games/combined_analyzed_games_cleaned.csv", index=False)

# Print the number of games removed
print(f"Number of games removed: {removed_games}")

Number of games removed: 11


# Creating Winning Chances column 

In [5]:

df['Evaluation'] = df['Evaluation'].astype(str).str.strip()
df['PlayerToMove'] = np.where(df['MoveNumber'] % 2 == 1, 'White', 'Black')

# Function to convert 'Evaluation' to 'New_evaluations'
def convert_evaluation(row):
    eval_str = row['Evaluation']
    
    if eval_str in ['+M0', '-M0', 'M0']:
        return 0.0  # Mate in 0 moves
    elif eval_str.startswith('+M') or (eval_str.startswith('M') and not eval_str.startswith('-M')):
        return 20.0  # White can mate
    elif eval_str.startswith('-M'):
        return -20.0  # Black can mate
    else:
        # Try to convert the evaluation to a float
        try:
            eval_float = float(eval_str)
            return eval_float  # Numeric evaluation remains the same
        except ValueError:
            return np.nan  # Unable to parse evaluation

# Apply the function to create 'New_evaluations' column
df['New_evaluations'] = df.apply(convert_evaluation, axis=1)

In [6]:
# Map 'Result' to outcome from White's perspective
def get_outcome(result):
    if result == '1-0':
        return 'Win'    # White won
    elif result == '0-1':
        return 'Loss'   # White lost
    elif result == '1/2-1/2':
        return 'Draw'   # Draw
    else:
        return None     # Exclude other results
    
    
def calculate_chances(df, lower_eval, upper_eval):
    # Filter positions where 'New_evaluations' is between lower_eval and upper_eval
    positions_in_range = df[(df['New_evaluations'] >= lower_eval) & (df['New_evaluations'] <= upper_eval)].copy()
    
    # Get unique GameIDs where this occurs
    games_in_range = positions_in_range['GameID'].unique()
    
    # Get the results of these games
    game_results = df[df['GameID'].isin(games_in_range)][['GameID', 'Result']].drop_duplicates()
    
    # Apply the mapping
    game_results['Outcome'] = game_results['Result'].apply(get_outcome)
    
    # Exclude games with 'Other' outcomes
    valid_results = game_results.dropna(subset=['Outcome'])
    
    # Total number of valid games
    total_valid_games = valid_results.shape[0]
    outcome_counts=None
    if total_valid_games == 0:
        winning_chance = drawing_chance = losing_chance = 0.0
    else:
        # Count the number of games in each category
        outcome_counts = valid_results['Outcome'].value_counts()
        
        # Calculate percentages
        winning_chance = (outcome_counts.get('Win', 0) / total_valid_games) * 100
        drawing_chance = (outcome_counts.get('Draw', 0) / total_valid_games) * 100
        losing_chance = (outcome_counts.get('Loss', 0) / total_valid_games) * 100
    
    return [winning_chance, drawing_chance, losing_chance, total_valid_games,outcome_counts]


calculate_chances(df,-21,-19)

[2.0408163265306123,
 1.0204081632653061,
 96.93877551020408,
 98,
 Outcome
 Loss    95
 Win      2
 Draw     1
 Name: count, dtype: int64]

In [8]:
results_df=pd.read_csv("winning_chances_adjusted.csv")
intervals = np.arange(-21, 21.5, 0.2)
intervals = np.round(intervals, decimals=1)
bin_labels = [f"({intervals[i]}, {intervals[i+1]}]" for i in range(len(intervals) - 1)]

# Bin 'New_evaluations' in 'df' to create an 'Interval' column
df['Interval'] = pd.cut(
    df['New_evaluations'],
    bins=intervals,
    labels=bin_labels,
    right=True,
    include_lowest=True,
)

# Merge 'df' with 'results_df' on 'Interval' to get 'WinningChance'
df = df.merge(results_df[['Interval', 'WinningChance', "LosingChance"]], on='Interval', how='left')

# Rename 'WinningChance' column to 'Winning_Chance' in 'df'
df.rename(columns={'WinningChance': 'Winning_Chance'}, inplace=True)


#df.to_csv("../huge_analyzed_games/combined_analyzed_15_16_winning_chances.csv")

# Calculate Chances based on Move Order

In [ ]:
# Your 'get_outcome' function
def get_outcome(result):
    if result == '1-0':
        return 'Win'    # White won
    elif result == '0-1':
        return 'Loss'   # White lost
    elif result == '1/2-1/2':
        return 'Draw'   # Draw
    else:
        return None     # Exclude other results
    
def calculate_chances(df, lower_eval, upper_eval, lower_move, upper_move):
    # Filter positions where 'New_evaluations' is between lower_eval and upper_eval
    # and 'MoveNumber' is between lower_move and upper_move
    positions_in_range = df[
        (df['Evaluation'] >= lower_eval) &
        (df['Evaluation'] <= upper_eval) &
        (df['MoveNumber'] >= lower_move) &
        (df['MoveNumber'] <= upper_move)
    ].copy()

    # Get unique GameIDs where this occurs
    games_in_range = positions_in_range['GameID'].unique()

    # Get the results of these games, ensuring one entry per GameID
    game_results = df[df['GameID'].isin(games_in_range)][['GameID', 'Result']].drop_duplicates(subset='GameID')

    # Apply the mapping
    game_results['Outcome'] = game_results['Result'].apply(get_outcome)

    # Exclude games with 'Other' outcomes
    valid_results = game_results.dropna(subset=['Outcome'])

    # Total number of valid games
    total_valid_games = valid_results.shape[0]

    outcome_counts = None
    if total_valid_games == 0:
        winning_chance = drawing_chance = losing_chance = 0.0
    else:
        # Count the number of games in each category
        outcome_counts = valid_results['Outcome'].value_counts()

        # Calculate percentages
        winning_chance = (outcome_counts.get('Win', 0) / total_valid_games) * 100
        drawing_chance = (outcome_counts.get('Draw', 0) / total_valid_games) * 100
        losing_chance = (outcome_counts.get('Loss', 0) / total_valid_games) * 100

    return [winning_chance, drawing_chance, losing_chance, total_valid_games, outcome_counts]


In [10]:
# Now, create move bins of 5 moves (assuming 'MoveNumber' increments by 1 per half-move)
# So each bin will cover 10 half-moves (5 full moves)
df=pd.read_csv("../huge_analyzed_games/combined_analyzed_games.csv")
# Define the maximum move number
max_move_number = df['MoveNumber'].max()

# Create move bins
move_bins = []
for i in range(0, int(max_move_number) + 10, 10):  # Increment by 10 half-moves
    lower_move = i + 1  # Start from 1
    upper_move = i + 10
    move_bins.append((lower_move, upper_move))

# Example evaluation interval
lower_eval = 4
upper_eval = 4.2

# Prepare a list to hold the results
results = []

# Loop over move bins and calculate chances
for bin_index, (lower_move, upper_move) in enumerate(move_bins, start=1):
    winning_chance, drawing_chance, losing_chance, total_valid_games, _ = calculate_chances(
        df, lower_eval, upper_eval, lower_move, upper_move
    )
    
    # Store the results
    results.append({
        'MoveBin': f"Bin {bin_index} ({lower_move}-{upper_move})",
        'WinningChance': winning_chance,
        'DrawingChance': drawing_chance,
        'LosingChance': losing_chance,
        'TotalGames': total_valid_games
    })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Creating the table (no need to run this yourselves)

In [ ]:
# Assume 'df' is your DataFrame and 'calculate_chances' function is already defined

# Define the intervals
intervals = np.arange(-20.2, 20.2, 0.2)
intervals = np.round(intervals, decimals=1)
# Prepare a list to hold the results
results = []

# Loop over intervals
for i in range(len(intervals) - 1):
    lower_eval = intervals[i]
    upper_eval = intervals[i + 1]
    
    # Call the calculate_chances function
    winning_chance, drawing_chance, losing_chance, total_valid_games = calculate_chances(df, lower_eval, upper_eval)[:4]
    
    # Store the results
    results.append({
        'Interval': f"({lower_eval}, {upper_eval}]",
        'LowerEval': lower_eval,
        'UpperEval': upper_eval,
        'WinningChance': winning_chance,
        'DrawingChance': drawing_chance,
        'LosingChance': losing_chance,
        'TotalGames': total_valid_games,
    })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Now, adjust the DataFrame to fill intervals with TotalGames == 0
# Find the closest interval with TotalGames > 0 and copy its chances

# Create a DataFrame of intervals with TotalGames > 0
non_zero_df = results_df[results_df['TotalGames'] > 0].reset_index(drop=True)

# Function to fill in chances for intervals with TotalGames == 0
def fill_chances(row):
    if row['TotalGames'] > 0:
        # Keep original values
        return row[['WinningChance', 'DrawingChance', 'LosingChance']]
    else:
        lower_eval = row['LowerEval']
        # Compute absolute difference in LowerEval
        diffs = (non_zero_df['LowerEval'] - lower_eval).abs()
        min_idx = diffs.idxmin()
        closest_row = non_zero_df.loc[min_idx]
        return closest_row[['WinningChance', 'DrawingChance', 'LosingChance']]

# Apply the function to fill in the missing chances
filled_chances = results_df.apply(fill_chances, axis=1)

# Assign the filled values back to the DataFrame
results_df[['WinningChance', 'DrawingChance', 'LosingChance']] = filled_chances

# Remove the 'LowerEval' and 'UpperEval' columns
results_df = results_df.drop(columns=['LowerEval', 'UpperEval'])

#results_df.to_csv('winning_chances.csv', index=False)

In [ ]:
# Adjust the 'WinningChance' column to be monotonically increasing
winning_chances = results_df['WinningChance'].values
for i in range(1, len(winning_chances)):
    if winning_chances[i] < winning_chances[i-1]:
        winning_chances[i] = winning_chances[i-1]
results_df['WinningChance'] = winning_chances

# Adjust the 'LosingChance' column to be monotonically decreasing
losing_chances = results_df['LosingChance'].values
for i in range(len(losing_chances)-2, -1, -1):
    if losing_chances[i] < losing_chances[i+1]:
        losing_chances[i] = losing_chances[i+1]
results_df['LosingChance'] = losing_chances

# Save the modified DataFrame back to CSV
results_df.to_csv('winning_chances_adjusted.csv', index=False)

In [21]:
import os
import numpy as np
import pandas as pd

# Assuming your DataFrame 'df' is loaded and contains the necessary columns

# Define the evaluation intervals
intervals = np.arange(-13, 13.2, 0.2)
intervals = np.round(intervals, decimals=1)
edges = [-np.inf] + list(intervals) + [np.inf]

# Create bin labels
bin_labels = []
for i in range(len(edges) - 1):
    lower = edges[i]
    upper = edges[i + 1]
    if np.isneginf(lower):
        label = f"(-infty, {upper}]"
    elif np.isposinf(upper):
        label = f"({lower}, infty)"
    else:
        label = f"({lower}, {upper}]"
    bin_labels.append(label)

# Set move range to cover all moves in the dataset
lower_move = df['MoveNumber'].min()
upper_move = df['MoveNumber'].max()

# Prepare a list to hold the results
results = []

# Loop over evaluation intervals
for i in range(len(edges) - 1):
    lower_eval = edges[i]
    upper_eval = edges[i + 1]

    # Call the calculate_chances function with move parameters covering all moves
    winning_chance, drawing_chance, losing_chance, total_valid_games, _ = calculate_chances(
        df, lower_eval, upper_eval, lower_move, upper_move
    )

    # Store the results
    results.append({
        'Interval': bin_labels[i],
        'WinningChance': winning_chance,
        'DrawingChance': drawing_chance,
        'LosingChance': losing_chance,
        'TotalGames': total_valid_games,
    })

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Save the DataFrame to a CSV file
output_filename = "winning_chances_all_moves.csv"
results_df.to_csv(output_filename, index=False)

print(f"Saved winning chances table for all moves to {output_filename}")


Saved winning chances table for all moves to winning_chances_all_moves.csv


# Creating The tables based on move order (no need to run this yourselves)

In [19]:
import os
import numpy as np
import pandas as pd
df=pd.read_csv("../huge_analyzed_games/combined_analyzed_games.csv")
#df=pd.read_csv("../Cleaned_Analyzed_Games/twic1477_16_processed.csv")

In [20]:
output_dir = '../test_winning_chances_tables'
os.makedirs(output_dir, exist_ok=True)

# Define the move bins
max_move_number = 120
move_range = 20
move_bins = []
for i in range(0, max_move_number, move_range):
    lower_move = i + 1
    upper_move = i + move_range
    move_bins.append((lower_move, upper_move))

# Define the evaluation intervals
intervals = np.arange(-13, 13.2, 0.2)
intervals = np.round(intervals, decimals=1)
edges = [-np.inf] + list(intervals) + [np.inf]

# Create bin labels
bin_labels = []
for i in range(len(edges) - 1):
    lower = edges[i]
    upper = edges[i + 1]
    if np.isneginf(lower):
        label = f"(-infty, {upper}]"
    elif np.isposinf(upper):
        label = f"({lower}, infty)"
    else:
        label = f"({lower}, {upper}]"
    bin_labels.append(label)

# Initialize variable to hold the reference table for moves 101-120
reference_results_df = None

# Loop over move bins up to 120
for lower_move, upper_move in move_bins:
    # Prepare a list to hold the results
    results = []

    # Loop over evaluation intervals
    for i in range(len(edges) - 1):
        lower_eval = edges[i]
        upper_eval = edges[i + 1]

        # Call the calculate_chances function with move bin parameters
        winning_chance, drawing_chance, losing_chance, total_valid_games, _ = calculate_chances(
            df, lower_eval, upper_eval, lower_move, upper_move
        )

        # Store the results
        results.append({
            'Interval': bin_labels[i],
            'WinningChance': winning_chance,
            'DrawingChance': drawing_chance,
            'LosingChance': losing_chance,
            'TotalGames': total_valid_games,
        })

    # Create a DataFrame from the results
    results_df = pd.DataFrame(results)

    # Save the DataFrame to a CSV file
    move_bin_label = f"{lower_move}-{upper_move}"
    output_filename = f"winning_chances_moves_{move_bin_label}.csv"
    output_path = os.path.join(output_dir, output_filename)
    results_df.to_csv(output_path, index=False)

    print(f"Saved winning chances table for moves {lower_move}-{upper_move} to {output_filename}")

    # If this is the move bin for moves 101-120, save the results_df as the reference table
    if lower_move == 101 and upper_move == 120:
        reference_results_df = results_df.copy()

# For move bins beyond 120, copy the table from moves 101-120
max_move_number_in_df = df['MoveNumber'].max()
additional_move_bins = []
current_move = 120
while current_move < max_move_number_in_df:
    lower_move = current_move + 1
    upper_move = current_move + move_range
    additional_move_bins.append((lower_move, upper_move))
    current_move += move_range

# Copy the reference table to additional move bins
if reference_results_df is not None:
    for lower_move, upper_move in additional_move_bins:
        # Save the reference DataFrame to a CSV file with the new move bin label
        move_bin_label = f"{lower_move}-{upper_move}"
        output_filename = f"winning_chances_moves_{move_bin_label}.csv"
        output_path = os.path.join(output_dir, output_filename)
        reference_results_df.to_csv(output_path, index=False)
        print(f"Copied winning chances table for moves {lower_move}-{upper_move} from moves 101-120")
else:
    print("Reference table for moves 101-120 is not available. Cannot copy to later move bins.")

Saved winning chances table for moves 1-20 to winning_chances_moves_1-20.csv
Saved winning chances table for moves 21-40 to winning_chances_moves_21-40.csv
Saved winning chances table for moves 41-60 to winning_chances_moves_41-60.csv
Saved winning chances table for moves 61-80 to winning_chances_moves_61-80.csv
Saved winning chances table for moves 81-100 to winning_chances_moves_81-100.csv
Saved winning chances table for moves 101-120 to winning_chances_moves_101-120.csv
Copied winning chances table for moves 121-140 from moves 101-120
Copied winning chances table for moves 141-160 from moves 101-120
Copied winning chances table for moves 161-180 from moves 101-120
Copied winning chances table for moves 181-200 from moves 101-120
Copied winning chances table for moves 201-220 from moves 101-120
Copied winning chances table for moves 221-240 from moves 101-120
Copied winning chances table for moves 241-260 from moves 101-120
Copied winning chances table for moves 261-280 from moves 10

In [12]:
df.GameID.nunique()

1174923

# Adding Winning Chances Column based on multiple tables 

In [1]:
import glob
import os
import pandas as pd
import numpy as np
import re
#df=pd.read_csv("../huge_analyzed_games/combined_analyzed_games.csv")
pd.set_option('display.max_columns', None)
df= pd.read_csv("../Cleaned_Analyzed_Games/twic920_15_processed.csv")

In [2]:


# Assume 'df' is your DataFrame with the analyzed games

# Read and combine all winning chances tables
winning_chances_tables = []
for filename in glob.glob('../winning_chances_tables/winning_chances_moves_*.csv'):
    # Extract the move bin from the filename
    basename = os.path.basename(filename)
    match = re.match(r'winning_chances_moves_(\d+)-(\d+)\.csv', basename)
    if match:
        lower_move = int(match.group(1))
        upper_move = int(match.group(2))
        move_bin_label = f"{lower_move}-{upper_move}"
        # Read the table
        results_df = pd.read_csv(filename)
        # Add the 'MoveBin' column
        results_df['MoveBin'] = move_bin_label
        # Append to the list
        winning_chances_tables.append(results_df)

# Combine all the winning chances tables
winning_chances_combined = pd.concat(winning_chances_tables, ignore_index=True)

# Prepare the main DataFrame 'df'

# Ensure 'MoveNumber' is numeric
df['MoveNumber'] = pd.to_numeric(df['MoveNumber'], errors='coerce')

# Define the move bins as in the winning chances tables
max_move_number = df['MoveNumber'].max()
bin_size = 10  # 10 half-moves per bin (5 full moves)

# Create edges for the bins
edges = list(range(1, int(max_move_number) + bin_size + 1, bin_size))
labels = [f"{edges[i]}-{edges[i+1]-1}" for i in range(len(edges)-1)]

# Assign 'MoveBin' labels to 'MoveNumber' in df
df['MoveBin'] = pd.cut(
    df['MoveNumber'],
    bins=edges,
    labels=labels,
    right=False,
    include_lowest=True
)

# Define evaluation intervals matching those in the winning chances tables
intervals = np.arange(-20.2, 20.2 + 0.2, 0.2)  # Include upper limit
intervals = np.round(intervals, decimals=1)
bin_labels = [f"({intervals[i]}, {intervals[i+1]}]" for i in range(len(intervals) - 1)]

# Bin 'New_evaluations' in 'df' to create an 'Interval' column
df['Interval'] = pd.cut(
    df['Evaluation'],
    bins=intervals,
    labels=bin_labels,
    right=True,
    include_lowest=True,
)

# Ensure 'MoveBin' and 'Interval' are strings for consistent merging
df['MoveBin'] = df['MoveBin'].astype(str)
df['Interval'] = df['Interval'].astype(str)
winning_chances_combined['MoveBin'] = winning_chances_combined['MoveBin'].astype(str)
winning_chances_combined['Interval'] = winning_chances_combined['Interval'].astype(str)

# Merge 'df' with 'winning_chances_combined' on 'MoveBin' and 'Interval' to get 'WinningChance' and 'LosingChance'
df = df.merge(
    winning_chances_combined[['MoveBin', 'Interval', 'WinningChance', 'LosingChance', 'TotalGames']],
    on=['MoveBin', 'Interval'],
    how='left'
)

# Now 'df' has the 'Winning_Chance' and 'LosingChance' columns added


In [3]:
df['WCL'] = df.groupby('GameID')['WinningChance'].diff().abs()
df['LCL'] = df.groupby('GameID')['LosingChance'].diff().abs()
df.loc[df['MoveNumber'] % 2 == 0, 'WCL'] = None
df.loc[df['MoveNumber'] % 2 != 0, 'LCL'] = None

# Adding Winning Chances Column based on ONE table

In [22]:
import glob
import os
import pandas as pd
import numpy as np
import re
#df=pd.read_csv("../huge_analyzed_games/combined_analyzed_games.csv")
pd.set_option('display.max_columns', None)
df= pd.read_csv("../Cleaned_Analyzed_Games/twic920_15_processed.csv")

In [23]:

# Step 1: Read the 'winning_chances_all_moves.csv' Table
results_df = pd.read_csv('winning_chances_all_moves.csv')

# Step 2: Define Evaluation Intervals and Bin Labels
# The intervals and bin labels should match those used when creating 'winning_chances_all_moves.csv'

# Recreate the edges and bin labels
intervals = np.arange(-13, 13.2, 0.2)
intervals = np.round(intervals, decimals=1)
edges = [-np.inf] + list(intervals) + [np.inf]

# Create bin labels
bin_labels = []
for i in range(len(edges) - 1):
    lower = edges[i]
    upper = edges[i + 1]
    if np.isneginf(lower):
        label = f"(-infty, {upper}]"
    elif np.isposinf(upper):
        label = f"({lower}, infty)"
    else:
        label = f"({lower}, {upper}]"
    bin_labels.append(label)

# Ensure that the bin labels in 'results_df' match the ones we're creating
# This is important for a correct merge
results_df['Interval'] = results_df['Interval'].astype(str)
bin_labels = [str(label) for label in bin_labels]

# Step 3: Bin the 'Evaluation' Values in 'df' to Create an 'Interval' Column
# Assuming 'df' is your analyzed chess DataFrame and 'Evaluation' column exists
df['Interval'] = pd.cut(
    df['Evaluation'],
    bins=edges,
    labels=bin_labels,
    right=True,
    include_lowest=True,
)

# Step 4: Merge 'df' with 'results_df' on 'Interval'
# Select the columns to merge
columns_to_merge = ['Interval', 'WinningChance', 'LosingChance', 'TotalGames']

# Ensure 'Interval' in 'df' is of type string
df['Interval'] = df['Interval'].astype(str)

# Perform the merge
df = df.merge(
    results_df[columns_to_merge],
    on='Interval',
    how='left'
)

# Now 'df' has the new columns added


In [25]:
df['WCL'] = df.groupby('GameID')['WinningChance'].diff().abs()
df['LCL'] = df.groupby('GameID')['LosingChance'].diff().abs()
df.loc[df['MoveNumber'] % 2 == 0, 'WCL'] = None
df.loc[df['MoveNumber'] % 2 != 0, 'LCL'] = None